# Python Data Testing Examples

This Jupyter Notebook is a tutorial for how to do Data Testing, or Data
Validation, in Python with examples using the `pandera`
[package](https://pandera.readthedocs.io/en/stable/index.html#).

The general idea of this package is to define the schema and expected constraints
and then check that the data set adheres to this schema and satisfies these constraints.

**PLEASE be vigilant when committing Jupyter Notebooks to GitLab.
Unlike R Markdown Notebooks, Jupyter Notebooks as saved as JSON files and
any dataframes you load into a Jupyter Notebook will be included in this JSON file.
Consequently, you should NEVER import patient information into a Jupyter Notebook
that you plan to commit since we cannot allow for PHI/PII to be saved in GitLab.
Any data from our data warehouse that you import into a Jupyter Notebook that will
be committed to GitLab should contain entirely deidentified information or
aggregated data ONLY. If you are using an Acuitas template to start your repo,
then .ipynb files will be included in your .gitignore file by default.**

To see an example of how a dataframe is stored in the Jupyter Notebook JSON file,
open the `PythonDataTestingExamples.ipynb` file in Notepad and search for "`valid_df`",
which is a data set we define below and then preview.

In [6]:
import numpy as np
import pandas as pd
import pandera as pa
from pandera.typing import DataFrame, Series

## Schema Models

First, we wil begin by defining a schema model for two different dataframes:
- One dataframe will be an input to a function
- Another dataframe will be an output of that same function

The schems are defined used `class` definitions.
Notice that `InputSchema` is passed as a parameter to the `OutputSchema`.
This means that `OutputSchema` will inherit all of the properties of
`InputSchema`.
We can see this from the print statement of each of these schema defintions.

In [7]:
class InputSchema(pa.SchemaModel):
    year: Series[int] = pa.Field(gt=2000, coerce=True)
    month: Series[int] = pa.Field(ge=1, le=12, coerce=True)
    day: Series[int] = pa.Field(ge=0, le=365, coerce=True)

class OutputSchema(InputSchema):
    revenue: Series[float]

print(InputSchema.to_schema())
print(OutputSchema.to_schema())

<Schema DataFrameSchema(
    columns={
        'year': <Schema Column(name=year, type=DataType(int64))>
        'month': <Schema Column(name=month, type=DataType(int64))>
        'day': <Schema Column(name=day, type=DataType(int64))>
    },
    checks=[],
    coerce=False,
    dtype=None,
    index=None,
    strict=False
    name=InputSchema,
    ordered=False,
    unique_column_names=False
)>
<Schema DataFrameSchema(
    columns={
        'year': <Schema Column(name=year, type=DataType(int64))>
        'month': <Schema Column(name=month, type=DataType(int64))>
        'day': <Schema Column(name=day, type=DataType(int64))>
        'revenue': <Schema Column(name=revenue, type=DataType(float64))>
    },
    checks=[],
    coerce=False,
    dtype=None,
    index=None,
    strict=False
    name=OutputSchema,
    ordered=False,
    unique_column_names=False
)>


Next, we will define our function that expects a dataframe with `InputSchema`
and returns a dataframe with `OutputSchema`.

In [8]:
@pa.check_types
def transform(df: DataFrame[InputSchema]) -> DataFrame[OutputSchema]:
    return df.assign(revenue=100.0)

### Valid Example

The `valid_df` DataFrame contains three columns of integers where the `Year`, `Month`, and `Day`
integer values are in the expected ranges.

In [9]:
valid_df = pd.DataFrame({
    "year": ["2001", "2002", "2003"],
    "month": ["3", "6", "12"],
    "day": ["200", "156", "365"],
})
valid_df

,year,month,day
0,2001,3,200
1,2002,6,156
2,2003,12,365


Applying the `transform` function to the `valid_df` DataFrame successfully returns a DataFrame with the following results:

In [10]:
transform(valid_df)

,year,month,day,revenue
0,2001,3,200,100.0
1,2002,6,156,100.0
2,2003,12,365,100.0


To validate the `valid_df` DataFrame outside of a function definition, we can use the `validate` function.

In [11]:
print(InputSchema.validate(valid_df))

   year  month  day
0  2001      3  200
1  2002      6  156
2  2003     12  365


### Invalid Example

The `invalid_df` DataFrame contains three columns of integers where the `Month` and `Day`
integer values are in the expected ranges, but the `Year` contains a value of "1999" where
we expect these values to be greater than 2000.

In [12]:
invalid_df = pd.DataFrame({
    "year": ["2001", "2002", "1999"],
    "month": ["3", "6", "12"],
    "day": ["200", "156", "365"],
})
invalid_df

,year,month,day
0,2001,3,200
1,2002,6,156
2,1999,12,365


Applying the `transform` function to the `invalid_df` DataFrame returns the following `SchemaError`
with a summary of the records that did not pass the validation tests:

In [13]:
transform(invalid_df)

SchemaError: error in check_types decorator of function 'transform': <Schema Column(name=year, type=DataType(int64))> failed element-wise validator 0:
<Check greater_than: greater_than(2000)>
failure cases:
   index  failure_case
0      2          1999

Using the `validate` function on the `invalid_df` DataFrame also returns a `SchemaError` with
a summary of the records that did not pass the validation tests:

In [14]:
print(InputSchema.validate(invalid_df))

SchemaError: <Schema Column(name=year, type=DataType(int64))> failed element-wise validator 0:
<Check greater_than: greater_than(2000)>
failure cases:
   index  failure_case
0      2          1999

## Custom Checks

Custom checks can be added as class methods to the Schema Model definitions.

For example, let's say we wanted to add a custom check to our `InputSchema` definition that
checked whether the product of the `month` and `day` integer values were greater than 1000. If not,
then the check fails.

In [32]:
class InputSchema(pa.SchemaModel):
    year: Series[int] = pa.Field(gt=2000, coerce=True)
    month: Series[int] = pa.Field(ge=1, le=12, coerce=True)
    day: Series[int] = pa.Field(ge=0, le=365, coerce=True)

    @pa.dataframe_check()
    def month_day_product_gt_1000(cls, df: pd.DataFrame) -> Series[bool]:
        return df['month'] * df['day'] > 1000

Validating the `valid_df` using this new definition for `InputSchema` with the `month_day_product_ft_1000`
check returns a `SchemaError` as expected with a print out of the rows that failed.

In [33]:
InputSchema.validate(valid_df)

SchemaError: <Schema DataFrameSchema(
    columns={
        'year': <Schema Column(name=year, type=DataType(int64))>
        'month': <Schema Column(name=month, type=DataType(int64))>
        'day': <Schema Column(name=day, type=DataType(int64))>
    },
    checks=[
        <Check month_day_product_gt_1000>
    ],
    coerce=False,
    dtype=None,
    index=None,
    strict=False
    name=InputSchema,
    ordered=False,
    unique_column_names=False
)> failed element-wise validator 0:
<Check month_day_product_gt_1000>
failure cases:
  column  index  failure_case
0   year      0          2001
1   year      1          2002
2  month      0             3
3  month      1             6
4    day      0           200
5    day      1           156